In [26]:
import torch
import torch_geometric


## Loading Dataset

In [27]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root = 'Citeseer', name = 'Citeseer')

In [28]:
data = dataset[0]
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


train mask is a binary mask (a tensor or array of True/False or 1/0 values) that indicates which nodes in the graph should be used during training.

In [29]:
print(f"Number of nodes: {data.num_nodes}")
print(f"Size of labels: {data.y.size(0)}")
print(f"Number of classes: {dataset.num_classes}")


Number of nodes: 3327
Size of labels: 3327
Number of classes: 6


In [30]:
data.y.unique()

tensor([0, 1, 2, 3, 4, 5])

## EDA

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
def visualize_graph(edge_index, labels, num_nodes):
    G = nx.Graph()
    edge_list = edge_index.t().tolist()  
    G.add_edges_from(edge_list)
    G.add_nodes_from(range(num_nodes)) 

    # Ensure labels match number of nodes
    labels = labels[:num_nodes]  # Truncate extra labels if necessary

    # Generate positions for the graph
    pos = nx.spring_layout(G, seed=42)  # Use spring layout for visualization

    plt.figure(figsize=(12, 8))
    nx.draw(
        G,
        pos,
        with_labels=False,
        node_color=labels,
        cmap=plt.cm.rainbow,
        node_size=50,
    )
    plt.title("Citeseer Graph Structure")
    plt.show()

# Call the function
visualize_graph(data.edge_index, data.y.numpy(), data.num_nodes)


In [31]:
print(data.edge_index)

tensor([[ 628,  158,  486,  ..., 2820, 1643,   33],
        [   0,    1,    1,  ..., 3324, 3325, 3326]])


## Model Training


$$
H^{(l+1)} = \sigma\left(\widehat{A} H^{(l)} W^{(l)}\right)
$$

$$
\widehat{A} = \widetilde{D}^{-1/2} \widetilde{A} \widetilde{D}^{-1/2} 
$$

In [32]:
class GCNLayer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x, edge_index):
        row, col = edge_index
        deg = torch.bincount(row, minlength=x.size(0))
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        out = torch.zeros_like(x)
        out.index_add_(0, row, x[col] * norm.unsqueeze(-1))

        # # Debug intermediate output
        # print(f"Intermediate output (after aggregation): {out.shape}") 

        # # Apply linear transformation
        out = self.linear(out) 
        # print(f"Output after linear transformation: {out.shape}

        return out


In [33]:
class GCN(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_features, hidden_features)
        self.gcn2 = GCNLayer(hidden_features, out_features)
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.LogSoftmax(dim=1)

    def forward(self, x, edge_index):
        x = self.gcn1(x, edge_index)
        x = self.relu(x)
        x = torch.nn.functional.dropout(x, training= self.training)
        x = self.gcn2(x, edge_index)
        return self.softmax(x)


In [34]:
def train(model, data, optimizer, criterion, train_mask):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data, mask):
    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index).argmax(dim=1)
        correct = (pred[mask] == data.y[mask]).sum()
        acc = int(correct) / int(mask.sum())
    return acc


In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(in_features=data.num_features, hidden_features=16, out_features=dataset.num_classes).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

In [36]:
for epoch in range(100):
    loss = train(model, data, optimizer, criterion, data.train_mask)
    train_acc = test(model, data, data.train_mask)
    val_acc = test(model, data, data.val_mask)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
    # if val_acc > .6500:
    #     break


Epoch 1, Loss: 1.7976, Train Acc: 0.4667, Val Acc: 0.1480
Epoch 2, Loss: 1.7105, Train Acc: 0.8167, Val Acc: 0.4700
Epoch 3, Loss: 1.5617, Train Acc: 0.9333, Val Acc: 0.5880
Epoch 4, Loss: 1.4159, Train Acc: 0.9583, Val Acc: 0.6160
Epoch 5, Loss: 1.2513, Train Acc: 0.9583, Val Acc: 0.6080
Epoch 6, Loss: 1.1661, Train Acc: 0.9667, Val Acc: 0.6000
Epoch 7, Loss: 1.0377, Train Acc: 0.9667, Val Acc: 0.6200
Epoch 8, Loss: 0.9195, Train Acc: 0.9750, Val Acc: 0.6380
Epoch 9, Loss: 0.7658, Train Acc: 0.9750, Val Acc: 0.6520
Epoch 10, Loss: 0.6638, Train Acc: 0.9750, Val Acc: 0.6620
Epoch 11, Loss: 0.5854, Train Acc: 0.9750, Val Acc: 0.6740
Epoch 12, Loss: 0.4582, Train Acc: 0.9833, Val Acc: 0.6740
Epoch 13, Loss: 0.4640, Train Acc: 0.9917, Val Acc: 0.6740
Epoch 14, Loss: 0.3727, Train Acc: 0.9917, Val Acc: 0.6680
Epoch 15, Loss: 0.2572, Train Acc: 0.9917, Val Acc: 0.6620
Epoch 16, Loss: 0.3034, Train Acc: 0.9917, Val Acc: 0.6620
Epoch 17, Loss: 0.2786, Train Acc: 0.9917, Val Acc: 0.6640
Epoch 

In [37]:
model.eval()
with torch.no_grad():
    pred1 = model(data.x, data.edge_index)
    pred = pred1.argmax(dim=1) 
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum() 
    acc = int(correct) / int(data.test_mask.sum())
    # f1 = f1_score(pred[data.test_mask].cpu(), data.y[data.test_mask].cpu(), average='macro') 
    print(f'Accuracy: {acc:.4f}') # ,f'F1_score: {f1:.4f}'


Accuracy: 0.6180


In [ ]:
def visualize_features(features, title="Node Features"):
    plt.figure(figsize=(12, 6))
    plt.imshow(features, aspect='auto', cmap='coolwarm')
    plt.colorbar()
    plt.title(title)
    plt.xlabel("Feature Dimension")
    plt.ylabel("Node Index")
    plt.show()

# visualize_features(data.x.numpy(), title="Original Node Features")

# Forward pass visualization
model = GCN(data.num_features, 16, dataset.num_classes)
with torch.no_grad():
    x = data.x
    visualize_features(x.numpy(), title="Input Features to Layer 1")
    x = model.gcn1(x, data.edge_index)
    visualize_features(x.numpy(), title="Output of GCN Layer 1")
    x = model.gcn2(x, data.edge_index)
    visualize_features(x.numpy(), title="Output of GCN Layer 2")


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_features, 16, dataset.num_classes).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

train_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(100):
    loss = train(model, data, optimizer, criterion, data.train_mask)
    train_acc = test(model, data, data.train_mask)
    val_acc = test(model, data, data.val_mask)

    train_losses.append(loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# Plot metrics
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label="Train Loss")
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.title("Training and Validation Metrics")
plt.legend()
plt.show()


In [ ]:
def visualize_adjacency(edge_index, num_nodes):
    """
    Visualizes the adjacency matrix of the graph.
    """
    adj_matrix = torch.zeros((num_nodes, num_nodes))
    adj_matrix[edge_index[0], edge_index[1]] = 1
    adj_matrix += torch.eye(num_nodes)
    plt.figure(figsize=(8, 8))
    plt.imshow(adj_matrix.numpy(), cmap='Greys', interpolation='none')
    plt.title("Adjacency Matrix")
    plt.colorbar(label="Edge Presence")
    plt.xlabel("Node")
    plt.ylabel("Node")
    plt.show()

visualize_adjacency(data.edge_index, data.num_nodes)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def visualize_embeddings(embeddings, labels, method='PCA', title='Node Embeddings'):
    """
    Visualize high-dimensional node embeddings using PCA or t-SNE.
    """
    if method == 'PCA':
        reducer = PCA(n_components=2)
    elif method == 't-SNE':
        reducer = TSNE(n_components=2, random_state=42)
    else:
        raise ValueError("Unsupported reduction method. Choose 'PCA' or 't-SNE'.")

    reduced_embeddings = reducer.fit_transform(embeddings)
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        reduced_embeddings[:, 0], reduced_embeddings[:, 1], 
        c=labels, cmap=plt.cm.rainbow, s=15
    )
    plt.colorbar(scatter, label="Node Labels")
    plt.title(title)
    plt.show()

# Forward pass to get node embeddings
model = GCN(data.num_features, 16, dataset.num_classes)
with torch.no_grad():
    embeddings1 = model.gcn1(data.x, data.edge_index)
    visualize_embeddings(embeddings1.numpy(), data.y.numpy(), method='t-SNE')

    embeddings2 = model.gcn2(embeddings1, data.edge_index)
    visualize_embeddings(embeddings2.numpy(), data.y.numpy(), method='t-SNE')


In [ ]:
def visualize_parameters(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            plt.figure()
            plt.hist(param.detach().cpu().numpy().flatten(), bins=50)
            plt.title(f"Distribution of {name}")
            plt.xlabel("Value")
            plt.ylabel("Frequency")
            plt.show()

visualize_parameters(model)


In [ ]:
# Save the trained model
torch.save(model.state_dict(), "gcn_model.pth")
print("Model saved to gcn_model.pth")


**Recreate the model architecture**

model = GCN(data.num_features, 16, dataset.num_classes)

model.load_state_dict(torch.load("gcn_model.pth"))

model.eval()  # Set the model to evaluation mode

print("Model loaded from gcn_model.pth")
